In [1]:
import soundfile as sf
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as sp
from numpy import fft

In [2]:
# Take in data path
# Return [spoof list, bonafide list]
def create_bf_spf_lists(path):
    f = open(path)
    spoof_list = []
    bonafide_list = []
    for line in f:
        tokens = line.strip().split()
        if tokens[3] == 'spoof':
            spoof_list.append(tokens[1])
        else:
            bonafide_list.append(tokens[1])
    return [spoof_list, bonafide_list]

[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.female.trl.txt")
[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.male.trl.txt")
'''
#eval instead
[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.female.trl.txt")
[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.male.trl.txt")
'''

'\n#eval instead\n[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\\LA\\LA\\ASVspoof2019_LA_asv_protocols\\ASVspoof2019.LA.asv.eval.female.trl.txt")\n[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\\LA\\LA\\ASVspoof2019_LA_asv_protocols\\ASVspoof2019.LA.asv.eval.male.trl.txt")\n'

In [3]:
def extract_time_data(list, num):
    time_data = []
    #fspoof = open(path, "w")
    for i in range(num):
        dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_dev\\flac\\' + list[i] + '.flac'
        #For eval instead
        #dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_eval\\flac\\' + list[i] + '.flac'
        x, sr = librosa.load(dpath, sr=44100)
        time_data.append(x)
    return (time_data, sr)

ftime_data_spoof, sr1 = extract_time_data(spoof_listf, 4500)
ftime_data_bonafide, sr2 = extract_time_data(bonafide_listf, 1500)
mtime_data_spoof, sr3 = extract_time_data(spoof_listm, 4500)
mtime_data_bonafide, sr4 = extract_time_data(bonafide_listm, 1500)

print(sr1)

In [ ]:
def get_max_length(data):
    return max(len(vector) for vector in data)

def pad_or_truncate(data, max_length):
    padded_data = []
    for vector in data:
        if len(vector) < max_length:
            pad_width = max_length - len(vector)
            padded_vector = np.pad(vector, pad_width=(0, pad_width), mode='constant')
        elif len(vector) > max_length:
            padded_vector = vector[:max_length]
        else:
            padded_vector = vector
        padded_data.append(padded_vector)
    return padded_data

# Extract mfccs from time_data with padding
def extract_mfccs(time_data, max_length, sr=44100, n_mels=20, hop_length=8192):
    padded_data = pad_or_truncate(time_data, max_length)
    
    mfccs_data = []
    
    for x in padded_data:
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mels=n_mels, hop_length=hop_length)
        # Flatten to a large vector

        #flattened_vector = np.array(mfccs).flatten()
        #mfccs_data.append(flattened_vector)
        #print(mfccs.shape)
        
        mfccs_data.append(mfccs)
    
    return np.array(mfccs_data)

fmax_length = max(get_max_length(ftime_data_spoof), get_max_length(ftime_data_bonafide))
mmax_length = max(get_max_length(mtime_data_spoof), get_max_length(mtime_data_bonafide))
max_length = max(fmax_length, mmax_length)

mfccs_spooff = extract_mfccs(ftime_data_spoof, max_length, sr=44100, n_mels=20, hop_length=16384)
mfccs_bonafidef = extract_mfccs(ftime_data_bonafide, max_length, sr=44100, n_mels=20, hop_length=16384)

mfccs_spoofm = extract_mfccs(mtime_data_spoof, max_length, sr=44100, n_mels=20, hop_length=16384)
mfccs_bonafidem = extract_mfccs(mtime_data_bonafide, max_length, sr=44100, n_mels=20, hop_length=16384)

In [ ]:
import numpy as np
import librosa

def pad_or_truncate(data, desired_length, sr):
    padded_data = []
    
    samples_per_second = sr
    
    for vector in data:
        current_length = len(vector)
        target_length = int(desired_length * samples_per_second)
        
        if current_length < target_length:
            pad_width = target_length - current_length
            padded_vector = np.pad(vector, pad_width=(0, pad_width), mode='constant')
        elif current_length > target_length:
            padded_vector = vector[:target_length]
        else:
            padded_vector = vector
        
        padded_data.append(padded_vector)
    
    return padded_data

def extract_mfccs(time_data, desired_length, sr=44100, n_mels=20, hop_length=8192):
    padded_data = pad_or_truncate(time_data, desired_length, sr)
    
    mfccs_data = []
    for x in padded_data:
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mels=n_mels, hop_length=hop_length)
        mfccs_data.append(mfccs)

        #flattened_vector = np.array(mfccs).flatten()
        #mfccs_data.append(flattened_vector)
    
    return np.array(mfccs_data)

# Assuming you have a desired length in seconds, e.g., 1 second
desired_length_seconds = 3

mfccs_spooff = extract_mfccs(ftime_data_spoof, desired_length_seconds, sr=44100, n_mels=30, hop_length=16384)
mfccs_bonafidef = extract_mfccs(ftime_data_bonafide, desired_length_seconds, sr=44100, n_mels=30, hop_length=16384)

mfccs_spoofm = extract_mfccs(mtime_data_spoof, desired_length_seconds, sr=44100, n_mels=30, hop_length=16384)
mfccs_bonafidem = extract_mfccs(mtime_data_bonafide, desired_length_seconds, sr=44100, n_mels=30, hop_length=16384)


(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)
(20, 9)


In [ ]:
np.set_printoptions(threshold=np.inf, precision=6, linewidth=np.inf, suppress=True)

# Open files for writing
with open("spoof_data_mfccs1_male.txt", "w") as fspoofw:
    for row in mfccs_spoofm:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_data_mfccs1_male.txt", "w") as fbonafidew:
    for row in mfccs_bonafidem:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')

with open("spoof_data_mfccs1_female.txt", "w") as fspoofw:
    for row in mfccs_spooff:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_data_mfccs1_female.txt", "w") as fbonafidew:
    for row in mfccs_bonafidef:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')

In [ ]:
print(mfccs_spooff.shape)
print(mfccs_bonafidef.shape)
print(mfccs_spoofm.shape)
print(mfccs_bonafidem.shape)

(4500, 20, 27)
(1500, 20, 27)
(4500, 20, 27)
(1500, 20, 27)


In [ ]:
import numpy as np

np.set_printoptions(threshold=np.inf, precision=6, linewidth=np.inf, suppress=True)

# Open file for writing spoof data for males
with open("spoof_data_mfccs1_male.txt", "w") as fspoofw:
    for matrix in mfccs_spoofm:
        np.savetxt(fspoofw, matrix.T, fmt='%f', delimiter=',')  # Transpose the matrix to be 27x20
        # Add a delimiter between matrices (optional)
        fspoofw.write('\n')

# Open file for writing bonafide data for males
with open("bonafide_data_mfccs1_male.txt", "w") as fbonafidew:
    for matrix in mfccs_bonafidem:
        np.savetxt(fbonafidew, matrix.T, fmt='%f', delimiter=',')  # Transpose the matrix to be 27x20
        # Add a delimiter between matrices (optional)
        fbonafidew.write('\n')

# Open file for writing spoof data for females
with open("spoof_data_mfccs1_female.txt", "w") as fspoofw:
    for matrix in mfccs_spooff:
        np.savetxt(fspoofw, matrix.T, fmt='%f', delimiter=',')  # Transpose the matrix to be 27x20
        # Add a delimiter between matrices (optional)
        fspoofw.write('\n')

# Open file for writing bonafide data for females
with open("bonafide_data_mfccs1_female.txt", "w") as fbonafidew:
    for matrix in mfccs_bonafidef:
        np.savetxt(fbonafidew, matrix.T, fmt='%f', delimiter=',')  # Transpose the matrix to be 27x20
        # Add a delimiter between matrices (optional)
        fbonafidew.write('\n')
